In [3]:
#Importing all the libraries
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import csv
import codecs
import cerberus
from schema import schema
import string
import requests
import sqlite3

In [54]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "LA.osm"  # Replace this with your osm file
SAMPLE_FILE = "sampleLA.osm"

k = 550 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [2]:
#OSM_FILE = "LA.osm"  
#SAMPLE_FILE = "sampleLA.osm"

#def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
#   context = iter(ET.iterparse(osm_file, events=('start', 'end')))
#  _, root = next(context)
# for event, elem in context:
#       if event == 'end' and elem.tag in tags:
#          yield elem
#         root.clear()

def count_tags(filename):    
    tags = {}
    
    for element in get_element(filename):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
        
    return tags
print "number of tags:"
count_tags(SAMPLE_FILE)

number of tags:


{'node': 567540, 'relation': 538, 'way': 56816}

In [3]:
#check to see if the element is a tag and if the k value is street
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib["k"] == "addr:street")

#check to see if the element is a tag and if the k value is a postal code
def is_postal_code(elem):
    return (elem.tag == "tag") and (elem.attrib["k"] == "addr:postcode")


### Looking at what is wrong with street names and postcodes

In [4]:
#print out the street names in our file
def printStreetnames():
    for event, elem in ET.iterparse(SAMPLE_FILE):
        if is_street_name(elem):
            print elem.attrib["v"]

printStreetnames()            

Kedge Ave
N Varney St
Frazier Street
Rinaldi Street
Pacific Avenue
South Ojai Street
Balboa Ave
Spruce St
Wilshire Boulevard
Buena Vista Dr
Charles Smith Ave
Syracuse St
Lakeshore St
Pattison St
23rd St
Arrow Hwy
5th St
Meadows Wy
Bittersweet Ln
Maddox Ct
Foothill Blvd
Dakota Ave
Cantabria Ave
Serena Pl
Clementine Dr
Wooden Gate Wy
Eastern Shore Dr
Carrollton Pl
Lexington Pl
Whitewater St
Alpine St
Cool Springs St
Wells Spring St
Morab St
Hungarian St
Altair Ct
Gertrudis Ct
Monte Vista Ave
Edison Ave
Fairview Dr
Lowell St
Ashford Ave
Sunnyview Dr
Lancewood Ave
Buxton St
Atlantic Blvd
Valley Blvd
Blanchard St
S Victoria Ave
Bluegrass Rd
Townsend Wy
Campus Wy
Wallace Wy
La Costa Ct
5th Ave
Red Rose Pl
10th Ave
North Vine Street
El Caminito Road
Merida Drive
Havencrest Drive
La Palma Drive
Rancho Heights Road
Olivos Court
Green Canyon Road
Toyon Heights Drive
Lake Circle Drive
Rainbow Glen Road
Birch Hill Road
Rice Canyon Road
Summit Avenue
East Elder Street
North Main Avenue
Hampton Road

In [5]:
#printing out the postal codes from my sample file with length greater than 5 because these are wrong
def printpostalcodes():
    for event, elem in ET.iterparse(SAMPLE_FILE):
        if is_postal_code(elem) and (len(elem.attrib['v']) > 5): #checking to see if the postal codes are greater than 5 chars
            print elem.attrib["v"]

printpostalcodes() 

CA 92509
90022-1601
CA 90638
90744-4522
91406-1232
90049-4013
91330-8449


### Fixing street types and postal codes

In [25]:
#audit postal code 
notpostcodes = []
def audit_postalcode(notpostcodes,value):
    m = postalcode_re.search(value)
    if not m:
        notpostcodes.add(value)
        
    return notpostcodes

In [64]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
postalcode_re = re.compile(r'^([0-9]){5}$')


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave." : "Avenue",
            "Av": "Avenue",
            "Av.": "Avenue",
            "Rd.": "Road",
            "Rd" : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Bd." : "Boulevard",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Ln" : "Lane",
            "Ln." : "Lane",
            "Wy" : "Way",
            "Ct" : "Court",
            "Hwy" : "Highway"
            }

#postalmapping = { "CA 92509" : "92509",
 #                 "90022-1601" : "90022",
#                "CA 90638" : "90638",
   #               "90744-4522": "90744",
    #              "91406-1232": "91406", 
     #             "90049-4013": "90049",
    #            "91330-8449" : "91330"
       #         }

#audit street type, and update name functions from quiz done earlier in the lesson

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    notpostalcodes = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):            
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_postal_code(tag):
                    audit_postalcode(notpostalcodes, tag.attrib['v'])
    osm_file.close()
    return street_types, notpostalcodes

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type, mapping[street_type], name)
    return name

def update_postcode(postalcode):
    if postalcode.isdigit():
        return postalcode
    elif re.findall("\CA ",postalcode):
        postalcode = postalcode.replace("CA ","").replace(" ","")
        return postalcode
    elif re.findall("\-",postalcode):
        postalcode = postalcode.split("-")[0]
        return postalcode
    else:
        postalcode = postalcode.replace(" ","")
        return postalcode


(street_types, wrongpostcodes) = audit(SAMPLE_FILE)

In [14]:
#printing out all of our street types 
pprint.pprint(dict(street_types))
print ""

{'A': set(['Via Estrada, Unit A']),
 'A-B': set(['West Manchester Avenue  Unit A-B']),
 'Abeto': set(['Abeto', 'Paseo Abeto']),
 'Ada': set(['Ada']),
 'Alamitos': set(['Alamitos']),
 'Alamos': set(['Calle de Los Alamos']),
 'Alaska': set(['Alaska']),
 'Albercon': set(['Avenida Albercon']),
 'Alder': set(['Alder', 'Speckled Alder']),
 'Alegria': set(['Alegria']),
 'Alessandro': set(['West Avenida Alessandro']),
 'Alevera': set(['Alevera']),
 'Algodon': set(['Avenida Algodon']),
 'Alisos': set(['Camino Alisos']),
 'Allegheny': set(['Allegheny']),
 'Alley': set(['Miller Alley']),
 'Altamira': set(['Via Altamira']),
 'Amapola': set(['Avenida Amapola']),
 'Amberhill': set(['Amberhill']),
 'Amigos': set(['Los Amigos']),
 'Antietam': set(['Antietam']),
 'Antonio': set(['West Avenida San Antonio']),
 'Anzio': set(['Anzio']),
 'Appleton': set(['Appleton']),
 'Aragon': set(['Avenida Aragon']),
 'Arboleda': set(['Paseo Arboleda']),
 'Ardmore': set(['Ardmore']),
 'Asbury': set(['Asbury']),
 'Ash':

In [15]:
#check to see if our update name function works
print ""
for st_type, ways in street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name    


West Canada => West Canada
East Canada => East Canada
La Perla => La Perla
Shelby => Shelby
Bumblebee => Bumblebee
Interlude => Interlude
Capri => Capri
Vanderbilt => Vanderbilt
Via Monserate => Via Monserate
Indiana => Indiana
Cypresswood => Cypresswood
Montecilo => Montecilo
Spyrock => Spyrock
Oakdale => Oakdale
Da Vinci => Da Vinci
Camino de Nog => Camino de Nog
Mecklenberg => Mecklenberg
Speckled Alder => Speckled Alder
Alder => Alder
Looking Glass => Looking Glass
Honeyflower => Honeyflower
Atlanta => Atlanta
Ellistone => Ellistone
Odyssey => Odyssey
Alaska => Alaska
Elmdale => Elmdale
Field Poppy => Field Poppy
Bright Poppy => Bright Poppy
Oak Barrel => Oak Barrel
Tranquil Heights => Tranquil Heights
Toscany => Toscany
Medford => Medford
Triana => Triana
West Marquita => West Marquita
Brandywine => Brandywine
Avenida Lobeiro => Avenida Lobeiro
Sagecrest => Sagecrest
Cienega => Cienega
Allegheny => Allegheny
Windsor => Windsor
Riveroaks => Riveroaks
Bayside => Bayside
Antietam =>

In [39]:
#check to see what are the incorrect postal codes within our file
print ""
print wrongpostcodes


set(['CA 92509', '91330-8449', '90744-4522', '90022-1601', 'CA 90638', '90049-4013', '91406-1232'])


In [65]:
#check to see if update postal code function works
print ""
for pc in wrongpostcodes:
        betterpostcode = update_postcode(pc)
        print pc, "=>", betterpostcode


CA 92509 => 92509
91330-8449 => 91330
90744-4522 => 90744
90022-1601 => 90022
CA 90638 => 90638
90049-4013 => 90049
91406-1232 => 91406


In [66]:
OSM_PATH = "sampleLA.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    waynodes = {}
    waynode = {}
    way_nodes = []
    idlist = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs = {key: element.attrib[key] for key in element.attrib if key in node_attr_fields}
        parent_id = element.attrib['id']
        for child in element.getchildren():
            k_value = child.attrib['k']

            # If problematic chars, move to the next child
            if re.search(problem_chars, k_value):
                continue

            if "addr:" in child.attrib['k']:
                if len((child.attrib['k'].split(":"))) < 3:
                    kval = child.attrib["k"].split(":", 1)[1]
                    if kval == "street":                        
                        child.attrib["v"] = update_name(child.attrib['v'],mapping)
                        
            if "addr:" in child.attrib['k']:
                if len((child.attrib['k'].split(":"))) < 3:
                    kval = child.attrib["k"].split(":", 1)[1]
                    if kval == "postcode":                        
                        child.attrib["v"] = update_postcode(child.attrib['v'])            
                    
            tag = {'value': child.attrib['v'], 'id': parent_id} 
            
            # Do the complicated colon string magic here
            if k_value.find(':') != -1:
                # First split the string with ":"
                k_split_by_colon = k_value.split(':')
                # Then update the tag dictionary
                tag.update({
                   'key': ':'.join(k_split_by_colon[1:]),
                   'type': k_split_by_colon[0]
                })
            else:
                # If no colon, just insert the original k value
                tag.update({
                    'key': k_value,
                    'type': 'regular'
                })
            # Append to tags
            tags.append(tag)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        way_attribs = {key: element.attrib[key] for key in element.attrib if key in way_attr_fields}
        way_parent_id = element.attrib['id']
        for way_child in element.getchildren():
            if way_child.tag == "nd":
                    new = {}
                    new['id'] = element.attrib["id"]
                    new['node_id'] = way_child.attrib["ref"]
                    new['position'] = len(way_nodes)
                    way_nodes.append(new)
            elif way_child.tag == "tag":
                way_k_value = way_child.attrib['k']
                if re.search(problem_chars, way_k_value):
                    continue
                    
                if "addr:" in way_child.attrib['k']:
                    if len((way_child.attrib['k'].split(":"))) < 3:
                        kval = way_child.attrib["k"].split(":", 1)[1]
                        if kval == "street":                        
                            way_child.attrib["v"] = update_name(way_child.attrib['v'],mapping)
                            
                if "addr:" in way_child.attrib['k']:
                    if len((way_child.attrib['k'].split(":"))) < 3:
                        kval = way_child.attrib["k"].split(":", 1)[1]
                        if kval == "postcode":                        
                            way_child.attrib["v"] = update_postcode(way_child.attrib['v'])           
                        
                tag2 = {'value': way_child.attrib['v'], 'id': way_parent_id}
    
                if way_k_value.find(':') != -1:
                    k_split_by_colon = way_k_value.split(':')
                    tag2.update({
                        'key': ':'.join(k_split_by_colon[1:]),
                        'type': k_split_by_colon[0]
                        })
                else:
                    tag2.update({
                        'key': way_k_value,
                        'type': 'regular'
                        })
                tags.append(tag2)            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
                    
                
            
            


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)                    
                    

### Taking the cleaned data and putting in a database

In [4]:
#initializing the database
db = sqlite3.connect("C:\Users\Krishna\Desktop\udacity projects\p3\p3project\p3.db")
c = db.cursor()

#### creating our tables within the database and populating them with values

In [70]:
#create the node table
node_table = """CREATE TABLE IF NOT EXISTS node( 
                        id INTEGER primary key,
                        lat REAL,
                        lon REAL,
                        user TEXT,
                        uid INTEGER,
                        version TEXT,
                        changeset INTEGER,
                        timestamp TEXT
                        );"""

c.execute(node_table)

In [105]:
query = "DELETE FROM node;"
c.execute(query)

with open(NODES_PATH,'rb') as source:
    nodesdict = csv.DictReader(source)
    inserting = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in nodesdict]
    
    c.executemany("INSERT INTO node(id, lat, lon, user, uid, version, changeset, timestamp) VALUES(?,?,?,?,?,?,?,?);", inserting)
    
    db.commit()

In [106]:
#lets see if something's actually in our table
Q1 = "SELECT * FROM node LIMIT 2"
c.execute(Q1)
rows = c.fetchall()
pprint.pprint(rows)

[(653656,
  33.9598406,
  -118.3770539,
  u'techlady',
  104962,
  u'10',
  12626769,
  u'2012-08-05T21:43:42Z'),
 (653814,
  34.0679644,
  -118.0361393,
  u'AM909',
  82317,
  u'18',
  6996057,
  u'2011-01-17T04:36:38Z')]


In [74]:
#create the node_tags table
node_tags_table = """CREATE TABLE IF NOT EXISTS node_tags(
                            id INTEGER references node(id),
                            key TEXT,
                            value TEXT,
                            type TEXT
                            );"""

c.execute(node_tags_table)

In [102]:
clear_query = "DELETE FROM node_tags;"
c.execute(clear_query)

with open(NODE_TAGS_PATH,'rb') as source:
    nodesdict = csv.DictReader(source)
    inserting = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in nodesdict]
    
    c.executemany("INSERT INTO node_tags(id, key, value, type) VALUES(?,?,?,?);", inserting)
    
    db.commit()


In [126]:
#lets see if something's actually in our table
Q2 = "SELECT * FROM node_tags LIMIT 2"
c.execute(Q2)
rows = c.fetchall()
pprint.pprint(rows)

[(14749652, u'highway', u'traffic_signals', u'regular'),
 (14840394, u'ref', u'105A', u'regular')]


In [123]:
droptable = "DROP TABLE IF EXISTS way"
c.execute(droptable)

#create the Way table
way_table = """CREATE TABLE IF NOT EXISTS way(
                        id INTEGER primary key,
                        user TEXT,
                        uid INTEGER,
                        version TEXT,
                        changeset INTEGER,
                        timestamp TEXT
                        );"""

c.execute(way_table)

In [125]:
clear_query = "DELETE FROM way;"
c.execute(clear_query)

with open(WAYS_PATH,'rb') as source:
    nodesdict = csv.DictReader(source)
    inserting = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in nodesdict]
    
    c.executemany("INSERT INTO way(id, user, uid, version,changeset,timestamp) VALUES(?,?,?,?,?,?);", inserting)
    
    #c.executemany("INSERT INTO way(id, user, uid, version, changeset, timestamp) VALUES(?,?,?,?,?,?);", inserting)
    
    db.commit()

In [127]:
#lets see if something's actually in our table
Q3 = "SELECT * FROM way LIMIT 2"
c.execute(Q3)
rows = c.fetchall()
pprint.pprint(rows)

[(3360891, u'RichRico', 2219338, u'8', 37297759, u'2016-02-18T21:16:50Z'),
 (3947263,
  u'florinbadita_telenav',
  2959379,
  u'15',
  34610721,
  u'2015-10-13T12:40:17Z')]


In [77]:
#create the way_nodes table
way_nodes_table = """CREATE TABLE IF NOT EXISTS way_nodes(
                            id INTEGER references way(id),
                            node_id INTEGER,
                            position INTEGER
                            );"""
c.execute(way_nodes_table)

In [113]:
clear_query = "DELETE FROM way_nodes;"
c.execute(clear_query)

with open(WAY_NODES_PATH,'rb') as source:
    nodesdict = csv.DictReader(source)
    inserting = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in nodesdict]
    
    c.executemany("INSERT INTO way_nodes(id, node_id, position) VALUES(?,?,?);", inserting)
    
    db.commit()

In [128]:
#lets see if something's actually in our table
Q4 = "SELECT * FROM way_nodes LIMIT 2"
c.execute(Q4)
rows = c.fetchall()
pprint.pprint(rows)

[(3360891, 16588169, 0), (3360891, 663129138, 1)]


In [80]:
#create the way_tags table
way_tags_table = """CREATE TABLE IF NOT EXISTS way_tags(
                            id INTEGER references way(id),
                            key TEXT,
                            value TEXT,
                            type TEXT
                            );"""
c.execute(way_tags_table)

In [114]:
clear_query = "DELETE FROM way_tags;"
c.execute(clear_query)

with open(WAY_TAGS_PATH,'rb') as source:
    nodesdict = csv.DictReader(source)
    inserting = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in nodesdict]
    
    c.executemany("INSERT INTO way_tags(id, key, value, type) VALUES(?,?,?,?);", inserting)
    
    db.commit()

In [129]:
#lets see if something's actually in our table
Q5 = "SELECT * FROM way_tags LIMIT 2"
c.execute(Q5)
rows = c.fetchall()
pprint.pprint(rows)

[(3360891, u'hov', u'lane', u'regular'), (3360891, u'lanes', u'2', u'regular')]


### Exploring the data

In [139]:
q = "PRAGMA page_size;"
c.execute(q)
rows = c.fetchall()
print "size of db: "
pprint.pprint(rows)

size of db: 
[(1024,)]


In [140]:
q = "PRAGMA page_count;"
c.execute(q)
rows = c.fetchall()
print "pages in db: "
pprint.pprint(rows)

pages in db: 
[(75708,)]


In [150]:
#number of unique users
q = "SELECT COUNT(DISTINCT(users.uid)) FROM (SELECT uid FROM node UNION ALL SELECT uid FROM way) users;"
c.execute(q)
rows = c.fetchall()
print "number of unique users in db: "
print rows

number of unique users in db: 
[(1619,)]


In [49]:
#top contributing users
q = "SELECT user, COUNT(*) as num FROM (SELECT user FROM node UNION ALL SELECT user FROM way) 
        users group by user order by num desc limit 10;"
c.execute(q)
rows = c.fetchall()
print "10 most contributing users: "
pprint.pprint(rows)

10 most contributing users: 
[(u'manings_labuildings', 51272),
 (u'schleuss_imports', 42050),
 (u'Luis36995_labuildings', 41982),
 (u'calfarome_labuilding', 38442),
 (u'dannykath_labuildings', 32620),
 (u'RichRico_labuildings', 30791),
 (u'Jothirnadh_labuildings', 28089),
 (u'karitotp_labuildings', 23781),
 (u'yurasi_import', 22675),
 (u'ridixcr_import', 21955)]


In [6]:
#number of nodes and ways
q1 = "select count(*) from node;"
q2 = "select count(*) from way;"

c.execute(q1)
rows = c.fetchall()
print "number of nodes: "
print rows

c.execute(q2)
rows = c.fetchall()
print "number of ways: "
print rows

number of nodes: 
[(567540,)]
number of ways: 
[(56816,)]


In [41]:
#what is there to eat in this area?

q = "SELECT x.value, count(*) as number FROM (SELECT * FROM node_tags union all select * from way_tags) as x where x.key = 'cuisine' group by x.value order by number desc limit 10;"


c.execute(q)
rows = c.fetchall()
pprint.pprint(rows)

[(u'burger', 11),
 (u'mexican', 9),
 (u'pizza', 6),
 (u'sandwich', 5),
 (u'american', 4),
 (u'chinese', 3),
 (u'coffee_shop', 3),
 (u'japanese', 3),
 (u'italian', 2),
 (u'thai', 2)]


In [43]:
#what are some amenities within the area?
q = "SELECT x.value, count(*) as number FROM (SELECT * FROM node_tags union all select * from way_tags) as x where x.key = 'amenity' group by x.value order by number desc limit 10;"


c.execute(q)
rows = c.fetchall()
pprint.pprint(rows)

[(u'parking', 245),
 (u'school', 101),
 (u'place_of_worship', 81),
 (u'restaurant', 60),
 (u'fast_food', 42),
 (u'toilets', 27),
 (u'fuel', 26),
 (u'cafe', 20),
 (u'bank', 18),
 (u'bench', 16)]
